# Data PM25 - Data Preprocessing

Data PM25 :  

~ Hourly Data  
~ Fill Data Null (Fillna)  
~ Aggregation to Daily Data  
~ Merge Data to be 1 File

1. Read File in Directory Local PC

In [9]:
import os
from datetime import datetime

# Ganti ke path folder yang ingin dilihat
folder_path = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\dataset\all year"

# Pastikan folder ada
if not os.path.exists(folder_path):
    print("❌ Folder tidak ditemukan.")
else:
    print(f"📂 Daftar file di: {folder_path}\n")
    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)
        if os.path.isfile(filepath):
            size_kb = os.path.getsize(filepath) / 1024
            modified_time = datetime.fromtimestamp(os.path.getmtime(filepath))
            print(f"{filename:50} {size_kb:10.1f} KB    {modified_time}")

📂 Daftar file di: C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\dataset\all year

ISPU-DKI3-JAGAKARSA-2022-01-01-20250501204016.xls        10.0 KB    2025-05-01 20:40:16
ISPU-DKI3-JAGAKARSA-2022-01-02-20250501204018 (1).xls       10.0 KB    2025-05-01 20:40:18
ISPU-DKI3-JAGAKARSA-2022-01-02-20250501204018.xls        10.0 KB    2025-05-01 20:40:18
ISPU-DKI3-JAGAKARSA-2022-01-02-20250501204019.xls        10.0 KB    2025-05-01 20:40:19
ISPU-DKI3-JAGAKARSA-2022-01-05-20250501204020.xls        10.0 KB    2025-05-01 20:40:20
ISPU-DKI3-JAGAKARSA-2022-01-06-20250501204022.xls        10.0 KB    2025-05-01 20:40:22
ISPU-DKI3-JAGAKARSA-2022-01-07-20250501204023.xls        10.0 KB    2025-05-01 20:40:23
ISPU-DKI3-JAGAKARSA-2022-01-07-20250501204024.xls        10.0 KB    2025-05-01 20:40:24
ISPU-DKI3-JAGAKARSA-2022-01-07-20250501204025 (1).xls       10.0 KB    2025-05-01 20:40:25
ISPU-DKI3-JAGAKARSA-2022-01-07-20250501204025.xls        10.0 KB    2025-05-01 20:40:25
ISPU-

2. Import Library for Preprocessing Data

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, time
import glob
import warnings
from pyxlsb import open_workbook as open_xlsb
import xlrd

# Suppress warnings
warnings.filterwarnings('ignore')

3. Load Data for Input & Output File

In [22]:
# Path configuration
INPUT_PATH = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\dataset\all year"
OUTPUT_PATH = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\processed_data"
os.makedirs(OUTPUT_PATH, exist_ok=True)

4. Determine all Function needed

In [ ]:
def determine_ispu_status(value):
    """Determine ISPU status based on value"""
    if pd.isna(value):
        return None
    try:
        value = float(value)
        if 0 <= value <= 50:
            return 'BAIK'
        elif 51 <= value <= 100:
            return 'SEDANG'
        elif 101 <= value <= 199:
            return 'TIDAK SEHAT'
        elif 200 <= value <= 299:
            return 'SANGAT TIDAK SEHAT'
        elif value >= 300:
            return 'BERBAHAYA'
    except ValueError:
        return None
    return None

def read_problematic_xls(filepath):
    """Special handler for problematic XLS files"""
    try:
        wb = xlrd.open_workbook(filepath, ignore_workbook_corruption=True)
        sheet = wb.sheet_by_index(0)
        data = []
        for i in range(sheet.nrows):
            data.append(sheet.row_values(i))
        return pd.DataFrame(data[1:], columns=data[0])
    except:
        try:
            with open_xlsb(filepath) as wb:
                with wb.get_sheet(1) as sheet:
                    data = []
                    for row in sheet.rows():
                        data.append([item.v for item in row])
                return pd.DataFrame(data[1:], columns=data[0])
        except:
            return None

def parse_filename(filename):
    """Extract date from filename"""
    for delimiter in ['_', '-']:
        parts = filename.split(delimiter)
        if len(parts) >= 6:
            try:
                return datetime.strptime(f"{parts[3]}-{parts[4]}-{parts[5]}", "%Y-%m-%d").date()
            except:
                continue
    return None

def clean_column_names(df):
    """Standardize column names"""
    column_map = {}
    for col in df.columns:
        col_upper = str(col).strip().upper()
        if 'PM10' in col_upper and 'ISPU' in col_upper:
            column_map[col] = 'ISPU_PM10'
        elif ('PM2.5' in col_upper or 'PM25' in col_upper) and 'ISPU' in col_upper:
            column_map[col] = 'ISPU_PM2_5'
        elif 'SO2' in col_upper and 'ISPU' in col_upper:
            column_map[col] = 'ISPU_SO2'
        elif 'CO' in col_upper and 'ISPU' in col_upper:
            column_map[col] = 'ISPU_CO'
        elif 'O3' in col_upper and 'ISPU' in col_upper:
            column_map[col] = 'ISPU_O3'
        elif 'NO2' in col_upper and 'ISPU' in col_upper:
            column_map[col] = 'ISPU_NO2'
        elif 'WAKTU' in col_upper:
            column_map[col] = 'WAKTU'
        elif 'STATUS' in col_upper and 'BY' in col_upper:
            # Handle existing status columns
            if 'PM10' in col_upper:
                column_map[col] = 'STATUS_BY_PM10'
            elif 'PM2.5' in col_upper or 'PM25' in col_upper:
                column_map[col] = 'STATUS_BY_PM2_5'
            elif 'SO2' in col_upper:
                column_map[col] = 'STATUS_BY_SO2'
            elif 'CO' in col_upper:
                column_map[col] = 'STATUS_BY_CO'
            elif 'O3' in col_upper:
                column_map[col] = 'STATUS_BY_O3'
            elif 'NO2' in col_upper:
                column_map[col] = 'STATUS_BY_NO2'
        elif 'OVERALL' in col_upper and 'STATUS' in col_upper:
            column_map[col] = 'OVERALL_ISPU_STATUS'
    return column_map

def process_time_column(time_series):
    """Convert time formats"""
    time_formats = ['%H:%M:%S', '%H.%M.%S', '%H:%M', '%H.%M', '%I:%M %p', '%I.%M %p']
    for fmt in time_formats:
        try:
            return pd.to_datetime(time_series, format=fmt).dt.time
        except:
            continue
    return pd.to_datetime(time_series, errors='coerce').dt.time

def process_single_file(filepath):
    """Process a single file"""
    filename = os.path.basename(filepath)
    file_date = parse_filename(filename)
    if not file_date:
        print(f"⚠️ Tidak dapat ekstrak tanggal dari: {filename}")
        return None

    try:
        # Try reading with different engines
        try:
            df = pd.read_excel(filepath, engine='openpyxl')
        except:
            try:
                df = pd.read_excel(filepath, engine='xlrd')
            except:
                df = read_problematic_xls(filepath)
                if df is None:
                    print(f"⚠️ Gagal membaca file: {filename}")
                    return None

        # Clean column names
        column_map = clean_column_names(df)
        df.rename(columns=column_map, inplace=True)
        
        # Process time column
        if 'WAKTU' in df.columns:
            df['WAKTU'] = process_time_column(df['WAKTU'])
            df = df[df['WAKTU'].notna()]
        else:
            # Create default hourly time if no time column exists
            hours = [time(hour) for hour in range(24)]
            df['WAKTU'] = hours[:len(df)]
        
        # Create datetime column
        df['DATETIME'] = pd.to_datetime(file_date.strftime('%Y-%m-%d') + ' ' + df['WAKTU'].astype(str))
        
        # ISPU columns to process
        ispu_cols = ['ISPU_PM10', 'ISPU_PM2_5', 'ISPU_SO2', 'ISPU_CO', 'ISPU_O3', 'ISPU_NO2']
        
        # Convert to numeric and fill missing values with hourly mean
        for col in ispu_cols:
            if col in df.columns:
                # Convert to numeric, coercing errors to NaN
                df[col] = pd.to_numeric(df[col], errors='coerce')
                if pd.api.types.is_numeric_dtype(df[col]):
                    hourly_mean = df.groupby('WAKTU')[col].transform('mean')
                    df[col] = df[col].fillna(hourly_mean)
        
        return df
    
    except Exception as e:
        print(f"❌ Error processing {filename}: {str(e)}")
        return None

def create_daily_aggregation(hourly_df):
    """Create daily aggregation with proper status columns"""
    if hourly_df.empty:
        return pd.DataFrame()
    
    # ISPU columns to process
    ispu_cols = ['ISPU_PM10', 'ISPU_PM2_5', 'ISPU_SO2', 'ISPU_CO', 'ISPU_O3', 'ISPU_NO2']
    
    # Get only columns that exist in the dataframe
    existing_ispu_cols = [col for col in ispu_cols if col in hourly_df.columns]
    
    # Group by date and get max values for each ISPU parameter
    daily_agg = hourly_df.groupby(hourly_df['DATETIME'].dt.date)[existing_ispu_cols].max()
    daily_agg = daily_agg.reset_index()
    daily_agg.rename(columns={'DATETIME': 'TANGGAL'}, inplace=True)
    
    # Calculate status for each parameter
    status_cols = []
    for param in existing_ispu_cols:
        status_col = f'STATUS_BY_{param.split("_")[1]}'
        daily_agg[status_col] = daily_agg[param].apply(determine_ispu_status)
        status_cols.append(status_col)
    
    # Determine overall status (worst status of the day)
    if status_cols:
        status_order = ['BERBAHAYA', 'SANGAT TIDAK SEHAT', 'TIDAK SEHAT', 'SEDANG', 'BAIK']
        
        def get_worst_status(row):
            for status in status_order:
                if status in row[status_cols].values:
                    return status
            return None
        
        daily_agg['OVERALL_ISPU_STATUS'] = daily_agg.apply(get_worst_status, axis=1)
    
    # Reorder columns
    final_columns = ['TANGGAL'] + existing_ispu_cols + status_cols + ['OVERALL_ISPU_STATUS']
    return daily_agg[[col for col in final_columns if col in daily_agg.columns]]

def main():
    print("Memulai pemrosesan data ISPU...")
    
    # Process all files
    all_data = []
    failed_files = []
    
    for file in glob.glob(os.path.join(INPUT_PATH, "*.xls*")):
        processed_df = process_single_file(file)
        if processed_df is not None:
            all_data.append(processed_df)
            print(f"✅ Berhasil memproses: {os.path.basename(file)}")
        else:
            failed_files.append(os.path.basename(file))
    
    if not all_data:
        print("⚠️ Tidak ada data yang berhasil diproses")
        return
    
    # Combine all data
    combined_hourly = pd.concat(all_data, ignore_index=True)
    combined_hourly.sort_values('DATETIME', inplace=True)
    
    # Create daily aggregation
    daily_agg = create_daily_aggregation(combined_hourly)
    
    # Save results
    hourly_output = os.path.join(OUTPUT_PATH, "ispu_hourly_complete.csv")
    daily_output = os.path.join(OUTPUT_PATH, "ispu_daily_complete.csv")
    
    combined_hourly.to_csv(hourly_output, index=False, encoding='utf-8-sig')
    daily_agg.to_csv(daily_output, index=False, encoding='utf-8-sig')
    
    # Save list of failed files
    if failed_files:
        with open(os.path.join(OUTPUT_PATH, "failed_files.txt"), 'w', encoding='utf-8') as f:
            f.write("\n".join(failed_files))
    
    print("\n✅ Pemrosesan selesai!")
    print(f"- Data per jam tersimpan di: {hourly_output}")
    print(f"- Data harian tersimpan di: {daily_output}")
    print(f"\nStatistik data:")
    print(f"Jumlah record hourly: {len(combined_hourly)}")
    print(f"Jumlah record daily: {len(daily_agg)}")
    print(f"Jumlah file gagal diproses: {len(failed_files)}")
    print("\nContoh data harian:")
    print(daily_agg.head())

if __name__ == "__main__":
    # Install pyxlsb if not available
    try:
        import pyxlsb
    except ImportError:
        import subprocess
        subprocess.check_call(["pip", "install", "pyxlsb"])
    
    main()

Memulai pemrosesan data ISPU...
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2022-01-01-20250501204016.xls
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2022-01-02-20250501204018 (1).xls
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2022-01-02-20250501204018.xls
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2022-01-02-20250501204019.xls
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2022-01-05-20250501204020.xls
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
✅ Berhasil memproses: ISPU-DKI3-JAGAKARSA-2022-01-06-20250501204022.xls
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 

In [ ]:
INPUT_FILE_NAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\dataset\all year\ispu_daily_complete.csv"

df = pd.read_csv(INPUT_FILE_NAME)
df.head()
print("show result")

5. Fill Null Value (Interpolate Linear)

In [ ]:
# ==========================================================
# 1. KONFIGURASI DAN INPUT
# ==========================================================


INPUT_FILE_NAME = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\dataset\all year\ispu_daily_complete.csv"
OUTPUT_PATH = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\01. Data Praprocessing\processed_data"

# Kolom yang akan digunakan
DATE_COL = 'TANGGAL'
PM25_COL = 'ISPU_PM2_5'
STATUS_COL = 'STATUS_BY_PM2' 
OUTPUT_FILE_NAME = "PM25_ISPU_Daily_Imputed.xlsx"

# ==========================================================
# 2. PEMUATAN DATA & PERSIAPAN
# ==========================================================

print("Memuat data...")
# Menggunakan encoding 'latin-1' atau 'cp1252' jika ada masalah karakter di CSV
try:
    df = pd.read_csv(INPUT_FILE_NAME)
except UnicodeDecodeError:
    df = pd.read_csv(INPUT_FILE_NAME, encoding='latin-1')

# Pilih hanya kolom yang dibutuhkan
df_pm25 = df[[DATE_COL, PM25_COL, STATUS_COL]].copy()

# Konversi tanggal dan set sebagai index
df_pm25[DATE_COL] = pd.to_datetime(df_pm25[DATE_COL], errors='coerce')
df_pm25 = df_pm25.dropna(subset=[DATE_COL]) # Hapus baris dengan tanggal tidak valid
df_pm25 = df_pm25.set_index(DATE_COL)
df_pm25.sort_index(inplace=True)

# Tentukan rentang waktu lengkap dari data (dari tanggal terlama hingga terbaru)
start_date = df_pm25.index.min()
end_date = df_pm25.index.max()
full_date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Buat kerangka waktu lengkap dan gabungkan data PM2.5
df_full = pd.DataFrame(index=full_date_range)
df_full = df_full.join(df_pm25[[PM25_COL, STATUS_COL]], how='left')


# ==========================================================
# 3. ANALISIS DATA KOSONG
# ==========================================================

initial_nan_count = df_full[PM25_COL].isnull().sum()
total_rows = len(df_full)
print(f"Total baris dalam rentang waktu penuh: {total_rows}")
print(f"Jumlah nilai PM2.5 yang hilang (NULL) awalnya: {initial_nan_count}")
print(f"Persentase data hilang: {initial_nan_count / total_rows * 100:.2f}%")


# ==========================================================
# 4. IMPUTASI PM2.5 UNTUK KELENGKAPAN 100%
# ==========================================================

print("\nMelakukan Imputasi (Pengisian Nilai Kosong)...")

# Langkah 1: Interpolasi Linier (Mengisi gap di tengah)
# Ini adalah cara terbaik untuk menjaga tren data
df_full['PM25_Imputed'] = df_full[PM25_COL].interpolate(method='linear')

# Langkah 2: Forward Fill (Mengisi sisa NaN di awal series)
# Ini memastikan tidak ada lagi NaN yang tersisa
df_full['PM25_Imputed'] = df_full['PM25_Imputed'].ffill()

# Langkah 3: Backward Fill (Mengisi sisa NaN di akhir series, jika ada)
# Ini hanya untuk memastikan 100% terisi jika data valid pertama/terakhir NaN
df_full['PM25_Imputed'] = df_full['PM25_Imputed'].bfill()


# ==========================================================
# 5. FINALISASI DAN EKSPOR
# ==========================================================

# Kolom Status harus diisi dengan nilai yang paling sering muncul (Mode)
# Karena status adalah kualitatif, kami mengisi dengan status yang paling sering muncul di sekitar NaN
df_full[STATUS_COL] = df_full[STATUS_COL].fillna(df_full[STATUS_COL].mode()[0])

# Pilih dan rapikan kolom output
df_output = df_full.reset_index().rename(
    columns={'index': DATE_COL, 'PM25_Imputed': 'ISPU_PM2_5_IMPUTED'}
)

# Konversi Tanggal ke format YYYY-MM-DD (untuk Tableau)
df_output[DATE_COL] = df_output[DATE_COL].dt.strftime('%Y-%m-%d')
df_output['ISPU_PM2_5_IMPUTED'] = df_output['ISPU_PM2_5_IMPUTED'].astype(np.float64)

# Hapus kolom ISPU_PM2_5 asli dan pilih kolom final
df_output_final = df_output[[
    DATE_COL, 'ISPU_PM2_5_IMPUTED', STATUS_COL
]]


# Export ke XLSX
   # Save results
# daily_output = os.path.join(OUTPUT_PATH, OUTPUT_FILE_NAME)
df_output_final.to_excel(OUTPUT_FILE_NAME, index=False) 

remaining_nan = df_output_final['ISPU_PM2_5_IMPUTED'].isnull().sum()
final_total_rows = len(df_output_final)

print(f"\n========================================================")
print(f"✅ Proses Perapian Data Selesai.")
print(f"✅ Total baris harian lengkap: {final_total_rows}")
print(f"✅ Sisa nilai NULL di ISPU_PM2_5_IMPUTED: {remaining_nan}")
print(f"✅ File Excel disimpan sebagai: {OUTPUT_FILE_NAME}")
print(f"========================================================")